In [1]:
from os import mkdir
from os.path import exists, join, basename
from shutil import rmtree, copytree
from yaml import dump
from glob import glob

In [2]:
DATASET = "dataset-YOLO"
YAML = "data.yaml"
ORIGEM = join("Boundingbox", "test")
DESTINO = DATASET

In [3]:
if not exists(DATASET):
    mkdir(DATASET)
else:
    rmtree(DATASET)
    mkdir(DATASET)

In [4]:
def creat_yaml(names: dict = {"0": "FAMACHA"}):
    yaml_configs = {
        "train": "train/",
        "val": "valid/",
        "test": "test/",
        "names": names
    }
    return yaml_configs

yaml_configs = creat_yaml()


In [5]:
with open(join(DATASET, YAML), "w") as file:
    dump(yaml_configs, file)

In [6]:
def dataset_yolo(roboflow_dataset, output_dataset):
    
    if exists(roboflow_dataset):
        
        for folder in ["test", "train", "valid"]:
            item = join(roboflow_dataset, folder)
            
            if exists(item):
                copytree(item, join(output_dataset,folder))

dataset_yolo("Boundingbox", DATASET)

In [7]:
def txt_polygon2xyxy(file: str = "file.txt", image_size: int  = 640)-> list:
    """
    Converter um arquivo txt com dados de um polygono em valores xyxy validos para a yolo em formato de lista de floats
    """
    
    with open(file, 'r') as arquivo:

        linhas = arquivo.readlines()
        
        
        if len(linhas)> 0:
            
            string = ''.join([linha.replace('\n',' ') for linha in linhas])
            #print(string)
            
            polygons = string.split(" ")
            print(polygons)
            
            itens = list(map(float,polygons))

            xyxy = list(map(lambda x: x / image_size, itens))
        
            return [int(item) if item == 0 else item for item in xyxy]
        
        else:
            return []


    
result = txt_polygon2xyxy(join("Boundingbox","test","labels","img101_3_jpg.rf.87b67fba66baf1711aa4a2397f49050b.txt"))
result

['0', '1.1572265625000002', '0.6035156250000002', '0.34863281249999994', '-0.205078125', '-0.2353515625', '0.37890625000000006', '0.5732421875000002', '1.1875000000000002', '0', '1.1517228597722124', '0.6191674171753088', '0.3707715064969799', '-0.21299386090485786', '-0.2295317612635287', '0.35036766730115754', '0.551419592011704', '1.1825289453813241']


[0,
 0.0018081665039062504,
 0.0009429931640625004,
 0.0005447387695312499,
 -0.0003204345703125,
 -0.00036773681640625,
 0.0005920410156250001,
 0.0008956909179687504,
 0.0018554687500000003,
 0,
 0.001799566968394082,
 0.00096744908933642,
 0.000579330478901531,
 -0.0003328029076638404,
 -0.0003586433769742636,
 0.0005474494801580587,
 0.0008615931125182875,
 0.001847701477158319]

In [8]:
def xyxy2text(xyxy: list, output: str, filename: str)->None:
    """
    Converte uma lista de valores float de um polygono pre-processado em um arquivo.txt com seus respectivos valores
    """

    with open(join(output,filename), 'w') as arquivo:
        # Escreve a string de itens no arquivo
        if len(xyxy) == 1:
            arquivo.write(' '.join(list(map(str, xyxy))))
        elif len(xyxy) > 1:
            arquivo.write(str(xyxy[0]))
            for value in xyxy[1:]:
                if value == 0:
                    arquivo.write('\n')
                    arquivo.write(str(value))
                else:
                    arquivo.write(f' {value}')

    
    
xyxy2text(result,'',"test.txt")

In [9]:
def format_dataset(roboflow_dataset):
    
    if exists(roboflow_dataset):
        
        for folder in ["test", "train", "valid"]:
            labels = join(roboflow_dataset, folder, "labels")
            
            if exists(labels):
                files = glob(join(labels,"*.txt"))
                for file in files:
                    polygon = txt_polygon2xyxy(file)
                    xyxy2text(polygon,labels,basename(file))

format_dataset("dataset-YOLO")

['0', '1.016869249682741', '0.3360961294416243', '0.7481410612309642', '0.026025142766497533', '0.17387769670050762', '0.5237200586928935', '0.4426058851522843', '0.8337910453680203']
['0', '0.6216637364860211', '0.3856789752186339', '0.48872400495799473', '0.2702697577382593', '0.1866327812973421', '0.6182482558790112', '0.31957251282536847', '0.7336574733593858']
['0', '0.6539482243754957', '0.49737884739294225', '0.510094140315226', '0.3917360044111817', '0.2826038672184774', '0.7015099933088819', '0.4264579512787471', '0.8071528362906424']
['0', '0.719929610148515', '0.44636641398514865', '0.5285775061881189', '0.4031578743811882', '0.43807240099009903', '0.8039661974009903', '0.6294245049504951', '0.8471747370049507']
['0', '0.8401592548076923', '0.7052584134615385', '0.7734975961538462', '0.4608323317307693', '0.34876802884615377', '0.5766676682692309', '0.4154296875', '0.8210937500000001']
['0', '0.7257590129732301', '0.3942893067992021', '0.3501739091351954', '0.334713738604203